# Supplementary - Cutoff point in tectal geometry

Conducts analyses related to Supplementary Figure S10.

In [ ]:
import sys
sys.path.append("..")

from main import *

# Torch stuff
if torch.cuda.is_available():
    device = "cuda:0"
else:
    device = "cpu"
print(device)

# Part 1: Demo of simulation in tectum geometry

In [ ]:
# Loading data
vertices = np.load('../Files/tectum_vertices_right.npy') * 40
vertices = np.stack([vertices[:, 2], vertices[:, 1], vertices[:, 0]], axis=1)
vertices /= np.max(vertices)
vertices -= np.mean(vertices, axis=0)
eigenmodes = np.load('../Files/tectum_eigenmodes_right.npy')[1:]
tectum = Geometry(vertices, eigenmodes.T)

In [ ]:
# Dynamical parameters
params = {'N_neurons': 1500,
          'h': 0.1,
          'g': 3,
          'tau': 3,
          'dale': False
          }

In [ ]:
simulator = Simulator(tectum,
                      ChaoticRNN,
                      params)

simulator.integrate(1000, output=False)

In [ ]:
%matplotlib inline
simulator.imshow(vmin=0, vmax=1)

In [ ]:
%matplotlib qt
simulator.animate(alpha=0.75,
                  cmap='hot',
                  vmax=1.5)

# Part 2: Sweep of $h$ values in tectal geometry to establish linear relationship

In [ ]:
from scipy.ndimage import gaussian_filter1d

def filter_matrix_rows(matrix, sigma=1):
    if sigma != 0:
        filtered = np.copy(matrix)
        for i in range(filtered.shape[0]):
            filtered[i] = gaussian_filter1d(filtered[i], sigma=sigma)
        return filtered
    else:
        return matrix


def identify_mode_cutoff_points(diagonals_matrix, sigma=0):

    matrix = filter_matrix_rows(diagonals_matrix, sigma=sigma)
    
    h_cutoff_array, r_cutoff_array = [], []    
    for element in matrix:
        diff_array = []
        for i in range(1, len(matrix[0] + 1)):
            diff = element[i] - element[i - 1]
            diff_array.append(diff)
        max_diff = min(diff_array)
        cutoff_index = diff_array.index(max_diff)
        while diff_array[cutoff_index - 1] > 0 and diff_array[cutoff_index - 1] > 0:
            diff_array[cutoff_index] = 0
            max_diff = min(diff_array)
            cutoff_index = diff_array.index(max_diff)
        h_cutoff_array.append(cutoff_index)
        r_cutoff_array.append((element[cutoff_index] + element[cutoff_index + 1]) / 2)

    return np.array(h_cutoff_array), np.array(r_cutoff_array)

Results were computed prior on Compute Canada (a few hours of simulations). See code example in the `ComputeCanada/kernel_size_tectum/` folder.

In [ ]:
%matplotlib inline

In [ ]:
correlations = np.load('../ComputeCanada/kernel_size_tectum/mode_correlations_per_h_2500.npy')

In [ ]:
avg_correlations = np.mean(np.abs(correlations), axis=1)

diagonals = []
for m in avg_correlations:
    diagonals.append(np.diag(m))
diagonals = np.stack(diagonals, axis=1)

Plotting matrix diagonals, analogous to Figure 3**b**.

In [ ]:
plt.imshow(diagonals)

Evaluating linear relationship

In [ ]:
h_values = np.linspace(1, 200, 100, endpoint=True)
wavelengths = np.load('../Results/wavelengths_tectum.npy')

h_cutoffs, _ = identify_mode_cutoff_points(diagonals, sigma=1)
h_cutoffs = h_values[h_cutoffs]

In [ ]:
x = h_cutoffs[1:]
y = np.array(wavelengths[1:50])

In [ ]:
(a, b), CI_a, CI_b = linear_regression_with_confidence_interval(x, y)
lower, upper = get_linear_bounds(x, CI_a, CI_b)

In [ ]:
fig, ax = plt.subplots(figsize=(2, 2), dpi=300)
ax.scatter(x, y, s=2, color='black')
ax.spines[['top', 'right']].set_visible(False)
ax.plot(x, a * x + b, linewidth=1, color='red')
ax.fill_between(x, lower, upper, color='red', edgecolor='None', alpha=0.1)
ax.set_xlabel('$h$ cutoff')
ax.set_ylabel('Mode wavelength')

In [ ]:
np.save('../Results/supp_tectum_cutoff_x1.npy', x)
np.save('../Results/supp_tectum_cutoff_y1.npy', y)

In [ ]:
print(a, b)

# Part 3: Alternate linear relationship from piecewise linear fits

In [ ]:
from scipy.optimize import curve_fit

Excluding the early and late phases of the $h$ sweep in the curve fit to avoid 1. the dynamical onset phase, where the network is too sparse to sustain dynamics, and  2. the overextended connectivity radius $h$ where eigenmode-gradient correlations are mostly zero, even though the radius keeps increasing.

In [ ]:
i1, i2 = 20, 80 

In [ ]:
cutoffs = []

for i in range(i1, i2):
    
    diagonal = np.diag(avg_correlations[i])

    x = np.arange(len(diagonal))
    y = diagonal
    initial_guess = [30, -1, 0]
    params, _ = curve_fit(piecewise_linear, x, y, p0=initial_guess)
    cutoff = int(np.round(params[0]))

    cutoffs.append(cutoff)

In [ ]:
%matplotlib inline

In [ ]:
plt.plot(cutoffs)

In [ ]:
cutoff_wavelengths = wavelengths[:][np.array(cutoffs)]

In [ ]:
y = cutoff_wavelengths
x = h_values[i1:i2]

In [ ]:
(a, b), CI_a, CI_b = linear_regression_with_confidence_interval(x, y)
lower, upper = get_linear_bounds(x, CI_a, CI_b)

In [ ]:
fig, ax = plt.subplots(figsize=(2, 2), dpi=300)
ax.scatter(x, y, s=2, color='black')
ax.spines[['top', 'right']].set_visible(False)
ax.plot(x, a * x + b, linewidth=1, color='red')
ax.fill_between(x, lower, upper, color='red', edgecolor='None', alpha=0.1)
ax.set_xlabel('$h$ cutoff')
ax.set_ylabel('Mode wavelength')

In [ ]:
pearsonr(x, y)[0] ** 2

In [ ]:
print(a, b)

In [ ]:
np.save('../Results/supp_tectum_cutoff_x2.npy', x)
np.save('../Results/supp_tectum_cutoff_y2.npy', y)

# Rendering figure

In [ ]:
from visualization import *

In [ ]:
fig = PaperFigure(figsize=(7, 1.5), dpi=600)

fig.set_tick_length(1)
fig.set_font_size(6)
fig.add_background()

# Adding panels -------------------------------


fig.add_axes('linear1', (0, 0), 1.25, 1.25)
fig.add_axes('linear2', (1.75, 0), 1.25, 1.25)

fig.set_line_thickness(0.6)

# Content -------------------------------------

ax = fig.axes['linear1']
x = np.load('../Results/supp_tectum_cutoff_x1.npy')
y = np.load('../Results/supp_tectum_cutoff_y1.npy')
(a, b), CI_a, CI_b = linear_regression_with_confidence_interval(x, y)
lower, upper = get_linear_bounds(x, CI_a, CI_b)
ax.scatter(x, y, color='black', edgecolor='None', s=5)
ax.plot(x, a * x + b, linewidth=1, color='red')
ax.fill_between(x, lower, upper, color='red', edgecolor='None', alpha=0.1)
ax.spines[['top', 'right']].set_visible(False)
print(pearsonr(x, y)[0] ** 2)
print(a, b)
print(CI_a)
print(CI_b)

print('\n')

ax = fig.axes['linear2']
x = np.load('../Results/supp_tectum_cutoff_x2.npy')
y = np.load('../Results/supp_tectum_cutoff_y2.npy')
(a, b), CI_a, CI_b = linear_regression_with_confidence_interval(x, y)
lower, upper = get_linear_bounds(x, CI_a, CI_b)
ax.scatter(x, y, color='black', edgecolor='None', s=5)
ax.plot(x, a * x + b, linewidth=1, color='red')
ax.fill_between(x, lower, upper, color='red', edgecolor='None', alpha=0.1)
ax.spines[['top', 'right']].set_visible(False)
print(pearsonr(x, y)[0] ** 2)
print(a, b)
print(CI_a)
print(CI_b)

# Displaying ----------------------------------

fig.save('../Figures/supp_tectum_cutoff_incomplete.svg')
fig.show()